# 3. How to select appropriate no. of layers  

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import keras_tuner as kt

# Load the diabetes dataset from UCI repository
df = pd.read_csv("/kaggle/input/diabetes/diabetes.csv")

# Display the first few rows to verify the data
print(df.head())

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [10]:
# Assuming the last column is the target variable and all others are features
X = df.iloc[:, :-1]  # All columns except the last one
y = df.iloc[:, -1]   # Last column as the target

print("Shape of X:", X.shape)  # Should be (768, 8) if using the diabetes dataset
print("Shape of y:", y.shape)  # Should be (768,)


Shape of X: (768, 8)
Shape of y: (768,)


In [11]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X= scale.fit_transform(X)

In [24]:
from sklearn.model_selection import train_test_split

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)  # Should be (614, 8)
print("Shape of X_test:", X_test.shape)    # Should be (154, 8)


Shape of X_train: (614, 8)
Shape of X_test: (154, 8)


In [25]:
def create_model(hp):
    # Initialize Sequential model
    model = Sequential()
    
    # Adding input layer and first hidden layer
    model.add(Dense(72, activation='relu', input_dim=8))
    
    # Adding hidden layers based on a hyperparameter value
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(72, activation='relu'))

    # Adding output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    # Compiling the model
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [26]:
tuner = kt.RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=5
)

Reloading Tuner from ./untitled_project/tuner0.json


In [27]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [28]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 5
Score: 0.7705627679824829

Trial 1 summary
Hyperparameters:
num_layers: 4
Score: 0.7662337422370911

Trial 0 summary
Hyperparameters:
num_layers: 7
Score: 0.7489177584648132

Trial 3 summary
Hyperparameters:
num_layers: 2
Score: 0.7489177584648132

Trial 4 summary
Hyperparameters:
num_layers: 3
Score: 0.7402597665786743


In [29]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [33]:
# Get the best model
model = tuner.get_best_models(num_models=1)[0]

In [34]:
history = model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6,validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7351 - loss: 0.4960 - val_accuracy: 0.6818 - val_loss: 0.5449
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7754 - loss: 0.4701 - val_accuracy: 0.7662 - val_loss: 0.5364
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7893 - loss: 0.4647 - val_accuracy: 0.7338 - val_loss: 0.5470
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8103 - loss: 0.4024 - val_accuracy: 0.6883 - val_loss: 0.5606
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7962 - loss: 0.4317 - val_accuracy: 0.7078 - val_loss: 0.5670
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8268 - loss: 0.4034 - val_accuracy: 0.7597 - val_loss: 0.6271
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8083 - loss: 0.4159 - val_accuracy: 0.7597 - val_loss: 0.5687
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8020 - loss: 0.4114 - val_accuracy: 0.68

### Accuracy of Model

In [35]:
loss,accuracy = model1.evaluate(X_test,y_test)
print("Accuracy of model:- ",accuracy*100)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6999 - loss: 3.2010 
Accuracy of model:-  70.77922224998474


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
